In [7]:
# !pip install yfinance

In [8]:
import pandas as pd
import yfinance as yf
from datetime import datetime

In [9]:
start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2024-07-03  220.000000  221.550003  219.029999  221.550003  221.293854   
       2024-07-05  221.649994  226.449997  221.649994  226.339996  226.078308   
       2024-07-08  227.089996  227.850006  223.250000  227.820007  227.556610   
       2024-07-09  227.929993  229.399994  226.369995  228.679993  228.415604   
       2024-07-10  229.300003  233.080002  229.250000  232.979996  232.710632   

                     Volume  
Ticker Date                  
AAPL   2024-07-03  37369800  
       2024-07-05  60412400  
       2024-07-08  59085900  
       2024-07-09  48076100  
       2024-07-10  62627700  


In [14]:
df_reset = df.reset_index()
print("First few rows of the DataFrame after resetting the index:")
print(df_reset.head())
print(f"\nDataFrame shape: {df_reset.shape}")


First few rows of the DataFrame after resetting the index:
   index Ticker       Date        Open        High         Low       Close  \
0      0   AAPL 2024-07-03  220.000000  221.550003  219.029999  221.550003   
1      1   AAPL 2024-07-05  221.649994  226.449997  221.649994  226.339996   
2      2   AAPL 2024-07-08  227.089996  227.850006  223.250000  227.820007   
3      3   AAPL 2024-07-09  227.929993  229.399994  226.369995  228.679993   
4      4   AAPL 2024-07-10  229.300003  233.080002  229.250000  232.979996   

    Adj Close    Volume  
0  221.293854  37369800  
1  226.078308  60412400  
2  227.556610  59085900  
3  228.415604  48076100  
4  232.710632  62627700  

DataFrame shape: (256, 9)


In [11]:
import plotly.io as pio
print("Available renderers:", pio.renderers)


Available renderers: Renderers configuration
-----------------------
    Default renderer: 'vscode'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']



In [12]:
import plotly.express as px
fig = px.line(df, x='Date', 
              y='Close', 
              color='Ticker', 
              title="Stock Market Performance for the Last 3 Months")
fig.show('vscode')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [28]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show('vscode')

In [29]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
57  229.031999  222.911501
58  228.664999  223.755000
59  227.220999  224.228500
60  226.212999  224.440500
61  224.955000  224.633500

[62 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
186         NaN         NaN
187         NaN         NaN
188         NaN         NaN
189         NaN         NaN
190         NaN         NaN
..          ...         ...
243  185.545001  185.920000
244  184.861002  186.060501
245  183.032001  185.500001
246  181.218001  184.689501
247  179.408000  183.780501

[62 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
62          NaN         NaN
63          NaN         NaN
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
..          ...         ...
119  450.145996  453.13

In [30]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'], 
                  title=f"{ticker} Moving Averages")
    fig.show('vscode')

In [31]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='Ticker', 
              title='Volatility of All Companies')
fig.show('vscode')

In [33]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT', 
                 trendline='ols', 
                 title='Correlation between Apple and Microsoft')
fig.show('vscode')

**STRONG CORELATION**